In [1]:
import json
import os
import shutil
from tqdm import tqdm
from PIL import Image

# --- ⚠️ LÜTFEN SADECE BU YOLLARI KONTROL EDİN ⚠️ ---

# 1. Balon veri setinin ana klasörü (Bu klasörün içinde /train/ ve /val/ olmalıdır)
BASE_DIR = r"C:\Users\endererkaya\OneDrive\Desktop\Mini Project 2\data\balloon" 

# 2. Yeni YOLO Segmentasyon formatında çıktıların geleceği temiz klasör
OUTPUT_YOLO_SEG_DIR = r"C:\Users\endererkaya\OneDrive\Desktop\Mini Project 2\balloon_YOLO_SEG_OUTPUT"

# --- SABİT TANIMLAMALAR ---
CATEGORY_ID = 0 # Balon tek sınıf olduğu için YOLO ID'si 0'dır.

def convert_via_to_yolo_seg(split_name):
    """via_region_data.json dosyasından etiketleri okur ve YOLO Segmentasyon formatına dönüştürür."""
    print(f"\n--- {split_name.upper()} Segmentasyon Etiketleri Dönüştürülüyor ---")
    
    # Giriş yolları
    split_path = os.path.join(BASE_DIR, split_name)
    json_file = os.path.join(split_path, 'via_region_data.json')
    
    # Çıktı yolları (Resimleri tekrar kopyalamayalım, sadece etiketleri oluşturalım)
    output_label_dir = os.path.join(OUTPUT_YOLO_SEG_DIR, 'labels', split_name)
    os.makedirs(output_label_dir, exist_ok=True)

    if not os.path.exists(json_file):
        print(f"HATA: {json_file} bulunamadı.")
        return

    with open(json_file, 'r') as f:
        data = json.load(f)
    
    # Yeni etiket klasörünü oluşturmadan önce eskisini temizle (opsiyonel)
    # shutil.rmtree(output_label_dir, ignore_errors=True) 
    # os.makedirs(output_label_dir, exist_ok=True)


    for filename_with_size, v_data in tqdm(data.items(), desc=f"İşleniyor {split_name}"):
        
        filename = v_data['filename']
        img_path = os.path.join(split_path, filename)
        
        if not os.path.exists(img_path):
             continue

        # Resim boyutunu PIL ile alalım (Normalize etmek için gerekli)
        try:
            img = Image.open(img_path)
            img_w, img_h = img.size
        except Exception as e:
            print(f"UYARI: {filename} boyutu alınamadı. Atlanıyor.")
            continue
            
        
        # Etiketlerin kaydedileceği TXT dosyası
        label_file_path = os.path.join(output_label_dir, filename.replace('.jpg', '.txt'))
        
        # 2. Segmentasyon Etiketlerini oluştur
        with open(label_file_path, 'w') as out_f:
            
            if 'regions' in v_data and v_data['regions']:
                
                for region_id, region in v_data['regions'].items():
                    
                    if region['shape_attributes']['name'] == 'polygon':
                        
                        all_x = region['shape_attributes']['all_points_x']
                        all_y = region['shape_attributes']['all_points_y']
                        
                        # ⚠️ YOLO SEGMENTASYON FORMATI: ⚠️
                        # [class_id x1_norm y1_norm x2_norm y2_norm ... xn_norm yn_norm]
                        
                        segment_line = f"{CATEGORY_ID}"
                        
                        # Poligon koordinatlarını Normalize Etme
                        for x, y in zip(all_x, all_y):
                            x_norm = x / img_w
                            y_norm = y / img_h
                            segment_line += f" {x_norm:.6f} {y_norm:.6f}"
                            
                        segment_line += "\n"
                        out_f.write(segment_line)

        # Resimleri kopyalama (YOLO'nun okuması için gerekli)
        target_img_dir = os.path.join(OUTPUT_YOLO_SEG_DIR, 'images', split_name)
        os.makedirs(target_img_dir, exist_ok=True)
        target_img_path = os.path.join(target_img_dir, filename)
        shutil.copy2(img_path, target_img_path) # Resimler de yeni klasöre kopyalanıyor

    print(f"{split_name.upper()} segmentasyon dönüşümü bitti.")


# --- ÇALIŞTIRMA KISMI ---

if __name__ == "__main__":
    
    # Önceki çıktıları temizle
    if os.path.exists(OUTPUT_YOLO_SEG_DIR):
        shutil.rmtree(OUTPUT_YOLO_SEG_DIR)
        
    convert_via_to_yolo_seg('train')
    convert_via_to_yolo_seg('val')
    
    print("\n--- TÜM SEGMENTASYON DÖNÜŞÜMÜ BAŞARILI! ---")
    print(f"Yeni Segmentasyon Formatı Hazırlandı: {OUTPUT_YOLO_SEG_DIR}")


--- TRAIN Segmentasyon Etiketleri Dönüştürülüyor ---


İşleniyor train: 100%|████████████████████████████████████████████████████████████████| 61/61 [00:00<00:00, 153.56it/s]


TRAIN segmentasyon dönüşümü bitti.

--- VAL Segmentasyon Etiketleri Dönüştürülüyor ---


İşleniyor val: 100%|██████████████████████████████████████████████████████████████████| 13/13 [00:00<00:00, 189.72it/s]

VAL segmentasyon dönüşümü bitti.

--- TÜM SEGMENTASYON DÖNÜŞÜMÜ BAŞARILI! ---
Yeni Segmentasyon Formatı Hazırlandı: C:\Users\endererkaya\OneDrive\Desktop\Mini Project 2\balloon_YOLO_SEG_OUTPUT
